# Evolution including Virus Family hybrid Clustering based on artificially mutated K-mers

## Milestones

- [x] HDBSCAN github errors
    - need to find version without problems
    - if now finding one revert back to MA version
    - revert back to Masterthesis and update jupyter lab, git and ressource
- [x] better inclusion of R, N, ... in the kmer
    - implemented, maybe need adjustment by value
    - if frature of missing higher than threshold garbage
    - if not fill missing by possible constellations
- [x] evolution on reading frame
    - difficult with amino conservation ORF tracker necessary
        - e.g. BLOSUM etc.
    - nucleotide exchange values used now, instead of amino exchange
        - usage of Kimura's two-parameter model
        - alpha and beta of user choice
- [ ] stable parameters 
    - best would be algorithmic solution here
    - number of clusters
        - neighbors -> distance matrix -> kneedle algorithm -> epsilon
    - sample number
        - cluster number extraction algorithms -> sample 
    - alpha value (A -> G, C -> T)
    - beta value (...)

## Implementation Blueprint

![Class2](Clusterer.svg)

## Packages 

In [125]:
import numpy as np
import pandas as pd
import itertools as it
from Bio import SeqIO
from Bio.Seq import Seq
import math
import re
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import multiprocessing as mp
import hdbscan
import progressbar
import scipy.spatial.distance as ssd
from scipy.cluster import hierarchy

## Classes and Definitions

In [3]:
class Vectors(object):
    
    def __init__(self, k = 7, identifier = 0, split = None, quality = {'':0}, variable = 0.9, state = 1.0, alpha = 0.0, beta = 0.0, init = 0.0, procs = 4):
    
        self.k = k
        self.quality = quality
        self.identifier = identifier
        self.split = split
        self.variable = variable
        self.nucleotides = ['A', 'C', 'G', 'T']
        self.substit = dict.fromkeys(map(ord, self.nucleotides), None)
        self.exist = dict.fromkeys(map(''.join, it.product(self.nucleotides, repeat = self.k)), init)        
        self.col = len(self.exist.keys())
        self.state = state
        self.procs = procs
        self.exchange = {
            'A':['A'],
            'C':['C'],
            'G':['G'],
            'T':['T'],
            'R':['A', 'G'],
            'Y':['C', 'T'],
            'W':['A', 'T'],
            'S':['C', 'G'],
            'M':['A', 'C'],
            'K':['G', 'T'],
            'B':['G', 'C', 'T'],
            'H':['A', 'C', 'T'],
            'D':['A', 'G', 'T'],
            'V':['A', 'C', 'G'],
            'N':['A', 'C', 'G', 'T'],
        } 
        self.kimura = {
            'A':{'A':state, 'C':beta, 'G':alpha, 'T':beta,},
            'C':{'A':beta, 'C':state, 'G':beta, 'T':alpha,},
            'G':{'A':alpha, 'C':beta, 'G':state, 'T':beta,},
            'T':{'A':beta, 'C':alpha, 'G':beta, 'T':state,},
        }
    
    def countRows(self, infile):
        
        sequences = {}
        index = []
        row = 0
        for entry in SeqIO.parse(infile,'fasta'):
            
            #name = entry.description
            #header = entry.name.split(self.split)
            header = entry.description.split(self.split)
            sequence = str(entry.seq)
            missing = len(sequence.translate(self.substit))
            fracture = float(len(sequence)/missing) if missing else 0 
            #accession = header[0]#.split(':')[1]
            accession = header[self.identifier] if type(self.identifier) == int else re.search('.*' + self.identifier + '([^|]+).*', entry.description)[1]
            
            try:
                if all([re.match(i, header[self.quality[i]], re.IGNORECASE) for i in self.quality]) == True and fracture <= self.variable:
                    row += 1
                    sequences[accession] = sequence
                    index.append(accession)
            except:
                pass
        
        return(row, index, sequences)
    
    def calculateKmer(self, sequence):
        
        temporary = self.exist.copy()
        for i in range(len(sequence) - self.k + 1):
            kmer = sequence[i:i+self.k]
            main = map(''.join, it.product(*[self.exchange.get(j) for j in kmer]))

            for sub in main:
                for l, nuc in enumerate(sub):
                    for mut in self.nucleotides:
                        mutation = sub[:l] + mut + sub[l+1:]
                        temporary[mutation] += self.kimura[nuc][mut]

        vector = np.fromiter(temporary.values(), dtype = 'float32', count = self.col)/sum(temporary.values())
        temporary.clear()
        return(vector)
        
    def calculateFrequence(self, infile):
        
        row, index, sequences = self.countRows(infile)
        matrix = np.empty((row, self.col, ),dtype = 'float32')
        
        widgets = [' [', progressbar.Timer(format = 'elapsed time: %(elapsed)s'), '] ', progressbar.Bar('#'),' (', progressbar.ETA(), ') ', ]
        bar = progressbar.ProgressBar(max_value = len(index), widgets = widgets).start()

        with mp.Pool(self.procs) as pool:
            result = pool.imap(self.calculateKmer, map(lambda m: sequences[m], index))
            for pos, vector in enumerate(result):
                matrix[pos] = vector
                bar.update(pos)
            
        bar.finish()

        return(index, matrix)

- execution can still be faster ca. 15-20min for segment 4 is still slow
    - inclusion of mutation increased the runtime by factor 5-10
    - multiprocessing difficult to implement (dicts, fast calculation of single instances high overhang)
- all mutations and all unkown kmers (including e.g. Ns) are counted with state or respective alpha beta
    - maybe split value by their number

In [4]:
def Cluster(linkage, min_clust, num_clust):

    x = 0.0
    y = 1.0
    cluster = linkage.get_clusters(cut_distance = x, min_cluster_size = min_clust)
    n = cluster.max().item()
    
    while n != num_clust:

        if n < num_clust and n != -1:
            x = x - y
            y = y * 0.1

        else:
            x = x + y

        cluster = linkage.get_clusters(cut_distance = x, min_cluster_size = min_clust)
        n = cluster.max().item()
        
        if x != 0.0 and n == -1:
            print('Error.')
            break
        
    return(cluster)

In [98]:
def workerCentroid(subl2):
    
    tuple_min = (0, '')
    for name, row in subl2.iterrows():

        if not tuple_min[1]:
            dist_min = ssd.cdist([row], subl2, 'euclidean').mean()
            tuple_min = (dist_min, name)
        else:
            dist_min = ssd.cdist([row], subl2, 'euclidean').mean()
            if dist_min < tuple_min[0]:
                tuple_min = (dist_min, name)
                
    return(tuple_min[1])

In [145]:
def Centroid(framecluster, framel2):
    
    groups = framecluster.query('cluster != -1').groupby('cluster').groups
    
    with mp.Pool(procs) as pool:
        result = pool.imap(workerCentroid, map(lambda match: framel2.loc[match], groups.values()))
        centroid = pd.DataFrame([(index, i) for i, index in enumerate(result)], columns = ['accession', 'cluster']).set_index('accession')
        
    return(centroid)

In [152]:
def Linkage(framecentroid, framel2):
    
    subl2 = framel2.loc[framecentroid.index.tolist()]
    distance = ssd.cdist(subl2, subl2, 'euclidean')
    linkage = hierarchy.linkage(distance, method = 'single', metric = 'euclidean')
    
    return(linkage)

- needs some kind of error correction e.g. when only 4 sequences 60 clusters are impossible

## Main Pipeline

In [5]:
k = 7
split = '|'
quality = {'(?!^UNKNOWN_.*)':1}
#quality = {'':0}
identifier = 0
variable = 0.9
min_clust = 2
sample = 1
num_clust =  60
n_components = 50
procs = 16
state = 1.0
alpha = 0.01
beta = 0.005
init = 0.0

In [6]:
vectors = Vectors(k = k, identifier = identifier, split = split, quality = quality, variable = variable, state = state, alpha = alpha, beta = beta, init = init, procs = procs)
index, matrixl1 = vectors.calculateFrequence('degue.fasta')

 [elapsed time: 0:02:30] |##################################| (Time:  0:02:30) 


In [11]:
pca = PCA(n_components = 50)
matrixpca = pca.fit_transform(matrixl1)
variance = pca.explained_variance_ratio_.sum()

In [12]:
matrixl2 = normalize(matrixpca, norm='l2')

In [13]:
hdbinit = hdbscan.HDBSCAN(min_samples = sample, min_cluster_size = min_clust, gen_min_span_tree = True, metric = 'euclidean').fit(matrixl2)

In [14]:
cluster = Cluster(linkage, min_clust, num_clust) 

In [119]:
framecluster = pd.DataFrame(cluster, columns = ['cluster'], index = index)
framel2 = pd.DataFrame(normalize(matrixl1, norm='l2'), index = index)

In [120]:
framecentroid = Centroid(framecl, framel2)

In [117]:
unclustered = len(framecl.query('cluster == -1'))
clustered = len(index)

In [153]:
#linkage = hdbinit.single_linkage_tree_
#framelinkage = linkage.to_pandas().drop(['parent'], axis=1)#.set_index('parent', inplace = False)
linkage = Linkage(framecentroid, framel2)
framelinkage = pd.DataFrame(linkage, columns = ['left_child', 'right_child', 'distance', 'size'])

<ipython-input-152-af985b35d8d4>:5: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linkage = hierarchy.linkage(distance, method = 'single', metric = 'euclidean')


In [154]:
framecluster.to_csv('cluster.csv', index=True, header=True, sep=',', mode='w')
framelinkage.to_csv('linkage.csv', index=False, header=True, sep=',', mode='w')
framecentroid.to_csv('centroid.csv', index=True, header=True, sep=',', mode='w')

## Results

![Result](Result.svg)

- NO CHANGE of H7 and H15 and H4 and H14!
    - mixing of subtypes correct?

## Garbage Place